In [29]:
import os
import pandas as pd

directory = "/home/krishna/Desktop/wind_turbine/data/" #input('Please enter the folder directory path :')  # Replace with the actual directory path
#year_repo = input('Please enter the year :')

# List all files in the directory
files = os.listdir(directory)

turbine1_df = pd.DataFrame()
turbine2_df = pd.DataFrame()
turbine3_df = pd.DataFrame()
turbine4_df = pd.DataFrame()
turbine5_df = pd.DataFrame()
turbine6_df = pd.DataFrame()

turbines = [turbine1_df, turbine2_df, turbine3_df,turbine4_df, turbine5_df, turbine6_df ]


csv_1_files = [file for file in files if file.endswith('.csv') and file.startswith('Turbine_Data_Kelmarsh_1')]
csv_2_files = [file for file in files if file.endswith('.csv') and file.startswith('Turbine_Data_Kelmarsh_2')]
csv_3_files = [file for file in files if file.endswith('.csv') and file.startswith('Turbine_Data_Kelmarsh_3')]
csv_4_files = [file for file in files if file.endswith('.csv') and file.startswith('Turbine_Data_Kelmarsh_4')]
csv_5_files = [file for file in files if file.endswith('.csv') and file.startswith('Turbine_Data_Kelmarsh_5')]
csv_6_files = [file for file in files if file.endswith('.csv') and file.startswith('Turbine_Data_Kelmarsh_6')]

csv_i_files  = [csv_1_files,csv_2_files,csv_3_files,csv_4_files,csv_5_files,csv_6_files]


In [41]:
for csv_1_file in csv_1_files:
    turbine1_df = pd.DataFrame()
    df = pd.read_csv(f'/home/krishna/Desktop/wind_turbine/data/{csv_1_file}',on_bad_lines='skip')
    turbine1_df.append(df)
    print(turbine1_df)
    

/tmp/ipykernel_2611/3005492194.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  turbine1_df.append(df)


Empty DataFrame
Columns: []
Index: []


/tmp/ipykernel_2611/3005492194.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  turbine1_df.append(df)


Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []


/tmp/ipykernel_2611/3005492194.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  turbine1_df.append(df)


In [40]:

from darts import TimeSeries
from darts.utils.timeseries_generation import linear_timeseries

frequency = '10min'

# Assuming you have a pandas DataFrame with a column named 'timestamp' and a column named 'power'
# and you want to split the data into 80% training and 20% testing

# Convert your data to a Darts TimeSeries object
series = TimeSeries.from_dataframe(data_20, '# Date and time', 'Power (kW)', freq=frequency)

# Split the time series into training and testing sets
train_size = int(len(series) * 0.8)  # 80% of the data for training
train, test = series[:train_size], series[train_size:]

# Print the lengths of the train and test sets
print("Train length:", len(train))
print("Test length:", len(test))



Empty DataFrame
Columns: []
Index: []
